This notebook shows how to fine-tuning the Weebit dataset using a pretrained model.

This notebook borrows most of the code from the following notebook. Thank you.  
https://www.kaggle.com/rhtsingh/commonlit-readability-prize-roberta-torch-fit

### Load Data

In [ ]:
import pandas as pd
import numpy as np

train = pd.read_csv('../input/clrp-external/weebit_reextracted.tsv', delimiter='\t')
train["readability"] = train["readability"] - 2  # transform base to 0

In [ ]:
print(train.readability.value_counts())
train.head()

In [ ]:
num_classes = train["readability"].nunique()
print(num_classes)

### Create Folds

In [ ]:
from sklearn.model_selection import StratifiedKFold
def create_folds(data, num_splits):
    data["kfold"] = -1
    skf = StratifiedKFold(n_splits=num_splits, random_state=2021, shuffle=True)
    for f, (t_, v_) in enumerate(skf.split(X=data, y=data["readability"])):
        data.loc[v_, 'kfold'] = f
    return data
train = create_folds(train, num_splits=5)

### Import Dependencies

In [ ]:
import argparse
from glob import glob
import os
import matplotlib.pyplot as plt
import json
from collections import defaultdict
import gc
gc.enable()

from sklearn.metrics import f1_score

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import Adam
from torch.optim.optimizer import Optimizer
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import (
    Dataset, DataLoader, 
    SequentialSampler, RandomSampler
)
from transformers import AutoConfig
from transformers import (
    AdamW,
    get_cosine_schedule_with_warmup, 
    get_cosine_with_hard_restarts_schedule_with_warmup,
    get_linear_schedule_with_warmup,
)
from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import MODEL_FOR_TOKEN_CLASSIFICATION_MAPPING
from IPython.display import clear_output
from tqdm import tqdm, trange

device = "cuda" if torch.cuda.is_available() else "cpu"
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))

### Convert Examples (Excerpt) to Features

In [ ]:
def convert_examples_to_features(data, tokenizer, max_len, is_test=False):
    data = data.replace('\n', '')
    tok = tokenizer.encode_plus(
        data, 
        max_length=max_len, 
        truncation=True,
        return_attention_mask=True,
        return_token_type_ids=True
    )
    curr_sent = {}
    padding_length = max_len - len(tok['input_ids'])
    curr_sent['input_ids'] = tok['input_ids'] + ([tokenizer.pad_token_id] * padding_length)
    curr_sent['token_type_ids'] = tok['token_type_ids'] + \
        ([0] * padding_length)
    curr_sent['attention_mask'] = tok['attention_mask'] + \
        ([0] * padding_length)
    return curr_sent

### Dataset Retriever

In [ ]:
class DatasetRetriever(Dataset):
    def __init__(self, data, tokenizer, max_len, is_test=False):
        self.data = data
        self.texts = self.data.text.values
        self.targets = self.data.readability.values
        self.tokenizer = tokenizer
        self.is_test = is_test
        self.max_len = max_len
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, item):
        text, label = self.texts[item], self.targets[item]
        features = convert_examples_to_features(
            text, self.tokenizer, 
            self.max_len, self.is_test
        )
        return {
            'input_ids':torch.tensor(features['input_ids'], dtype=torch.long),
            'token_type_ids':torch.tensor(features['token_type_ids'], dtype=torch.long),
            'attention_mask':torch.tensor(features['attention_mask'], dtype=torch.long),
            'label':torch.tensor(label, dtype=torch.long),
        }

### Model

In [ ]:
class CommonLitModel(nn.Module):
    def __init__(
        self, 
        model_name, 
        config,
        num_classes=5,
        multisample_dropout=False,
        output_hidden_states=False,
        dropout_rate=0.0
    ):
        super(CommonLitModel, self).__init__()
        self.config = config
        self.model = AutoModel.from_pretrained(
            model_name, 
            output_hidden_states=output_hidden_states
        )
        self.layer_norm = nn.LayerNorm(config.hidden_size)
        if multisample_dropout:
            self.dropouts = nn.ModuleList([
                nn.Dropout(dropout_rate) for _ in range(5)
            ])
        else:
            self.dropouts = nn.ModuleList([nn.Dropout(dropout_rate)])
        self.classifier = nn.Linear(config.hidden_size, num_classes)
        self._init_weights(self.layer_norm)
        # self._init_weights(self.classifier)
 
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
 
    def forward(
        self, 
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        labels=None
    ):
        outputs = self.model(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        pooler_output = outputs.pooler_output  # (batch_size, hidden_size)
        pooler_output = self.layer_norm(pooler_output)
 
        # max-avg head
#         average_pool = torch.mean(sequence_output, 1)
#         max_pool, _ = torch.max(sequence_output, 1)
#         concat_sequence_output = torch.cat((average_pool, max_pool), 1)
 
        # multi-sample dropout
        for i, dropout in enumerate(self.dropouts):
            if i == 0:
                logits = self.classifier(dropout(pooler_output))
            else:
                logits += self.classifier(dropout(pooler_output))
        
        logits /= len(self.dropouts)
 
        # calculate loss
        loss = None
        if labels is not None:
            # regression task
            loss_fn = torch.nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels.view(-1))
        
        output = (logits,) + outputs[2:]
        return ((loss,) + output) if loss is not None else output

### Optimizer

In [ ]:
class Lamb(Optimizer):
    # Reference code: https://github.com/cybertronai/pytorch-lamb

    def __init__(
        self,
        params,
        lr: float = 1e-3,
        betas = (0.9, 0.999),
        eps: float = 1e-6,
        weight_decay: float = 0,
        clamp_value: float = 10,
        adam: bool = False,
        debias: bool = False,
    ):
        if lr <= 0.0:
            raise ValueError('Invalid learning rate: {}'.format(lr))
        if eps < 0.0:
            raise ValueError('Invalid epsilon value: {}'.format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError(
                'Invalid beta parameter at index 0: {}'.format(betas[0])
            )
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError(
                'Invalid beta parameter at index 1: {}'.format(betas[1])
            )
        if weight_decay < 0:
            raise ValueError(
                'Invalid weight_decay value: {}'.format(weight_decay)
            )
        if clamp_value < 0.0:
            raise ValueError('Invalid clamp value: {}'.format(clamp_value))

        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        self.clamp_value = clamp_value
        self.adam = adam
        self.debias = debias

        super(Lamb, self).__init__(params, defaults)

    def step(self, closure = None):
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                if grad.is_sparse:
                    msg = (
                        'Lamb does not support sparse gradients, '
                        'please consider SparseAdam instead'
                    )
                    raise RuntimeError(msg)

                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state['step'] = 0
                    # Exponential moving average of gradient values
                    state['exp_avg'] = torch.zeros_like(
                        p, memory_format=torch.preserve_format
                    )
                    # Exponential moving average of squared gradient values
                    state['exp_avg_sq'] = torch.zeros_like(
                        p, memory_format=torch.preserve_format
                    )

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                state['step'] += 1

                # Decay the first and second moment running average coefficient
                # m_t
                exp_avg.mul_(beta1).add_(grad, alpha=1 - beta1)
                # v_t
                exp_avg_sq.mul_(beta2).addcmul_(grad, grad, value=1 - beta2)

                # Paper v3 does not use debiasing.
                if self.debias:
                    bias_correction = math.sqrt(1 - beta2 ** state['step'])
                    bias_correction /= 1 - beta1 ** state['step']
                else:
                    bias_correction = 1

                # Apply bias to lr to avoid broadcast.
                step_size = group['lr'] * bias_correction

                weight_norm = torch.norm(p.data).clamp(0, self.clamp_value)

                adam_step = exp_avg / exp_avg_sq.sqrt().add(group['eps'])
                if group['weight_decay'] != 0:
                    adam_step.add_(p.data, alpha=group['weight_decay'])

                adam_norm = torch.norm(adam_step)
                if weight_norm == 0 or adam_norm == 0:
                    trust_ratio = 1
                else:
                    trust_ratio = weight_norm / adam_norm
                state['weight_norm'] = weight_norm
                state['adam_norm'] = adam_norm
                state['trust_ratio'] = trust_ratio
                if self.adam:
                    trust_ratio = 1

                p.data.add_(adam_step, alpha=-step_size * trust_ratio)

        return loss

### Discriminative Learning Rate and Weight Decay

In [ ]:
def get_optimizer_params(args, model):
    no_decay = ["bias", "LayerNorm.weight"]
    if args.layerwise_learning_rate_decay == 1.0:
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": args.weight_decay,
                "lr": args.lr,
            },
            {
                "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
                "lr": args.lr,
            },
        ]
    else:
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() if "classifier" in n or "pooler" in n],
                "weight_decay": 0.0,
                "lr": args.lr,
            },
        ]

        if args.model_type in ["bert", "roberta", "electra", "albert"]:
            num_layers = model.model.config.num_hidden_layers
            layers = [getattr(model, args.model_type).embeddings] + list(getattr(model, args.model_type).encoder.layer)
            layers.reverse()
            lr = args.lr
            for layer in layers:
                lr *= args.layerwise_learning_rate_decay
                optimizer_grouped_parameters += [
                    {
                        "params": [p for n, p in layer.named_parameters() if not any(nd in n for nd in no_decay)],
                        "weight_decay": args.weight_decay,
                        "lr": lr,
                    },
                    {
                        "params": [p for n, p in layer.named_parameters() if any(nd in n for nd in no_decay)],
                        "weight_decay": 0.0,
                        "lr": lr,
                    },
                ]
        else:
            raise NotImplementedError
    return optimizer_grouped_parameters

### Utils

In [ ]:
def make_model(model_name='roberta-large', tokenizer_name='roberta-large', num_labels=5):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    config = AutoConfig.from_pretrained(model_name)
    config.update({'num_labels': num_labels})
    model = CommonLitModel(
        model_name, 
        config=config,
        num_classes=num_labels,
        multisample_dropout=False,
    )
    model = model.to(device)
    return model, tokenizer

def make_optimizer(model, optimizer_name="AdamW"):
    kwargs = {
        'lr': 2e-5,
        'weight_decay': 0,
        'adam_epsilon': 1e-8,
        'layerwise_learning_rate_decay': 1.0,
        'model_type': 'roberta',
    }
    
    optimizer_grouped_parameters = get_optimizer_params(argparse.Namespace(**kwargs), model)
    if optimizer_name == "LAMB":
        optimizer = Lamb(optimizer_grouped_parameters, **kwargs)
        return optimizer
    elif optimizer_name == "Adam":
        optimizer = Adam(optimizer_grouped_parameters, **kwargs)
        return optimizer
    elif optimizer_name == "AdamW":
        optimizer = AdamW(
            optimizer_grouped_parameters, 
            lr=kwargs["lr"], 
            eps=kwargs["adam_epsilon"],
            correct_bias=True,
        )
        return optimizer
    else:
        raise Exception('Unknown optimizer: {}'.format(optimizer_name))

def make_scheduler(optimizer, decay_name='cosine_warmup', warmup_steps=0, t_max=10):
    if decay_name == 'step':
        scheduler = optim.lr_scheduler.MultiStepLR(
            optimizer,
            milestones=[30, 60, 90],
            gamma=0.1
        )
    elif decay_name == 'cosine':
        scheduler = lrs.CosineAnnealingLR(
            optimizer,
            T_max=t_max
        )
    elif decay_name == "cosine_warmup":
        scheduler = get_cosine_schedule_with_warmup(
            optimizer,
            num_warmup_steps=warmup_steps,
            num_training_steps=t_max
        )
    elif decay_name == "linear":
        scheduler = get_linear_schedule_with_warmup(
            optimizer, 
            num_warmup_steps=warmup_steps, 
            num_training_steps=t_max
        )
    else:
        raise Exception('Unknown lr scheduler: {}'.format(decay_type))    
    return scheduler    

def make_loader(
    data, 
    tokenizer, 
    max_len,
    batch_size,
    fold=0
):
    train_set, valid_set = data[data['kfold']!=fold], data[data['kfold']==fold]
    train_dataset = DatasetRetriever(train_set, tokenizer, max_len)
    valid_dataset = DatasetRetriever(valid_set, tokenizer, max_len)

    train_sampler = RandomSampler(train_dataset)
    train_loader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        sampler=train_sampler, 
        pin_memory=True, 
        drop_last=True,
        num_workers=4
    )

    valid_sampler = SequentialSampler(valid_dataset)
    valid_loader = DataLoader(
        valid_dataset, 
        batch_size=batch_size // 2, 
        sampler=valid_sampler, 
        pin_memory=True, 
        drop_last=False, 
        num_workers=4
    )

    return train_loader, valid_loader

### Metrics

In [ ]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        self.max = 0
        self.min = 1e5

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        if val > self.max:
            self.max = val
        if val < self.min:
            self.min = val

### Trainer

In [ ]:
class Trainer:
    def __init__(self, model, optimizer, scheduler, scalar=None, log_interval=1, evaluate_interval=1):
        self.model = model
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.scalar = scalar
        self.log_interval = log_interval
        self.evaluate_interval = evaluate_interval
        self.evaluator = Evaluator(self.model, self.scalar)

    def train(self, train_loader, valid_loader, epoch, 
              result_dict, tokenizer, fold):
        total_loss = 0
        count = 0
        losses = AverageMeter()
        
        for batch_idx, batch_data in enumerate(train_loader):
            self.model.train()
            
            input_ids, attention_mask, token_type_ids, labels = batch_data['input_ids'], \
                batch_data['attention_mask'], batch_data['token_type_ids'], batch_data['label']
            input_ids, attention_mask, token_type_ids, labels = \
                input_ids.to(device), attention_mask.to(device), token_type_ids.to(device), labels.to(device)

            
            if self.scalar is not None:
                with torch.cuda.amp.autocast():
                    outputs = self.model(
                        input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        labels=labels
                    )
            else:
                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids,
                    labels=labels
                )
            
            loss, logits = outputs[:2]
            total_loss += loss.tolist()
            count += labels.size(0)
            losses.update(loss.item(), input_ids.size(0))
            
            if self.scalar is not None:
                self.scalar.scale(loss).backward()
                self.scalar.step(self.optimizer)
                self.scalar.update()
            else:
                loss.backward()
                self.optimizer.step()
                
            self.scheduler.step()
            self.optimizer.zero_grad()
            
            if batch_idx % self.log_interval == 0:
                _s = str(len(str(len(train_loader.sampler))))
                ret = [
                    ('epoch: {:0>3} [{: >' + _s + '}/{} ({: >3.0f}%)]').format(epoch, count, len(train_loader.sampler), 100 * count / len(train_loader.sampler)),
                    'train_loss: {: >4.5f} lr: {}'.format(losses.avg, self.optimizer.param_groups[0]["lr"]),
                ]
                print(', '.join(ret))
            
            if batch_idx % self.evaluate_interval == 0:
                result_dict = self.evaluator.evaluate(
                    valid_loader, 
                    epoch, 
                    result_dict, 
                    tokenizer
                )
                if result_dict['f1_score'][-1] > result_dict['best_f1_score']:
                    print("{} epoch, best epoch was updated! valid_f1_score: {: >4.5f}".format(epoch, result_dict['f1_score'][-1]))
                    result_dict["best_val_loss"] = result_dict['val_loss'][-1]
                    result_dict["best_f1_score"] = result_dict['f1_score'][-1]
                    torch.save(self.model.state_dict(), f"model{fold}.bin")

        result_dict['train_loss'].append(losses.avg)
        return result_dict

### Evaluator

In [ ]:
class Evaluator:
    def __init__(self, model, scalar=None):
        self.model = model
        self.scalar = scalar
    
    def worst_result(self):
        ret = {
            'loss':float('inf'),
            'accuracy':0.0
        }
        return ret

    def result_to_str(self, result):
        ret = [
            'epoch: {epoch:0>3}',
            'loss: {loss: >4.2e}'
        ]
        for metric in self.evaluation_metrics:
            ret.append('{}: {}'.format(metric.name, metric.fmtstr))
        return ', '.join(ret).format(**result)

    def save(self, result):
        with open('result_dict.json', 'w') as f:
            f.write(json.dumps(result, sort_keys=True, indent=4, ensure_ascii=False))
    
    def load(self):
        result = self.worst_result
        if os.path.exists('result_dict.json'):
            with open('result_dict.json', 'r') as f:
                try:
                    result = json.loads(f.read())
                except:
                    pass
        return result

    def evaluate(self, data_loader, epoch, result_dict, tokenizer):
        losses = AverageMeter()

        self.model.eval()
        total_loss = 0
        predictions = []
        targets = []
        with torch.no_grad():
            for batch_idx, batch_data in enumerate(data_loader):
                input_ids, attention_mask, token_type_ids, labels = batch_data['input_ids'], \
                    batch_data['attention_mask'], batch_data['token_type_ids'], batch_data['label']
                input_ids, attention_mask, token_type_ids, labels = input_ids.to(device), \
                    attention_mask.to(device), token_type_ids.to(device), labels.to(device)
                
                if self.scalar is not None:
                    with torch.cuda.amp.autocast():
                        outputs = self.model(
                            input_ids=input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            labels=labels
                        )
                else:
                    outputs = self.model(
                        input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        labels=labels
                    )
                
                loss, logits = outputs[:2]
                losses.update(loss.item(), input_ids.size(0))
                preds = torch.argmax(logits, dim=1)
                predictions.append(preds.detach().cpu().numpy())
                targets.append(labels.detach().cpu().numpy())
        predictions = np.concatenate(predictions)
        targets = np.concatenate(targets)
        f1 = f1_score(targets, predictions, average='weighted')
        print('----Validation Results Summary----')
        print('Epoch: [{}] valid_loss: {: >4.5f} f1 score: {}'.format(epoch, losses.avg, f1))

        result_dict['val_loss'].append(losses.avg)
        result_dict['f1_score'].append(f1)     
        return result_dict

### Config

In [ ]:
def config(fold=0):
    seed = 2021
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    epochs = 5
    max_len = 512
    batch_size = 8
    warmup_ratio = 0.0

    model_name = 'roberta-base'  # roberta-base roberta-large albert-base-v2
    tokenizer_name = model_name

    model, tokenizer = make_model(
        model_name=model_name, 
        tokenizer_name=tokenizer_name, 
        num_labels=num_classes,
    )
    train_loader, valid_loader = make_loader(
        train, tokenizer, max_len=max_len,
        batch_size=batch_size, fold=fold
    )
    
    num_update_steps_per_epoch = len(train_loader)
    max_train_steps = epochs * num_update_steps_per_epoch
    if warmup_ratio > 0:
        warmup_steps = int(warmup_ratio * num_update_steps_per_epoch)
    else:
        warmup_steps = 0
        
    optimizer = make_optimizer(model, "AdamW")
    scheduler = make_scheduler(
        optimizer,
        decay_name='cosine_warmup',
        warmup_steps=warmup_steps, 
        t_max=max_train_steps
    )

    # scaler = torch.cuda.amp.GradScaler()
    scaler = None

    result_dict = {
        'epoch':[], 
        'train_loss': [], 
        'val_loss' : [], 
        'test_loss': [],
        'best_val_loss': np.inf,
        'f1_score': [],
        'best_f1_score': 0,
    }
    return (
        model, tokenizer, 
        optimizer, scheduler, 
        scaler, train_loader, 
        valid_loader, result_dict, 
        epochs
    )

### Run

In [ ]:
def run(fold=0):
    model, tokenizer, optimizer, scheduler, scaler, \
        train_loader, valid_loader, result_dict, epochs = config(fold)
    import time
    trainer = Trainer(model, optimizer, scheduler, scaler)
    train_time_list = []

    for epoch in range(epochs):
        result_dict['epoch'] = epoch

        tic1 = time.time()

        result_dict = trainer.train(train_loader, valid_loader, epoch, 
                                    result_dict, tokenizer, fold)

        tic2 = time.time() 
        train_time_list.append(tic2 - tic1)

    torch.cuda.empty_cache()
    del model, tokenizer, optimizer, scheduler, \
        scaler, train_loader, valid_loader,
    gc.collect()
    
    return result_dict

### Train

In [ ]:
result_list = []
for fold in range(5):
    print('----')
    print(f'FOLD: {fold}')
    result_dict = run(fold)
    result_list.append(result_dict)
    print('----')
    print()

In [ ]:
[print("FOLD::", i, "Loss:: ", fold['best_val_loss']) for i, fold in enumerate(result_list)]

In [ ]:
[print("FOLD::", i, "F1:: ", fold['best_f1_score']) for i, fold in enumerate(result_list)]